<a href="https://colab.research.google.com/github/MiguelEgidoMorales/UFV-visualizacion/blob/main/ejercicio_de_clase/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### EJERCICIO CLASE


Cargamos el archivo csv, y lo guardamos usando ';' en vez de ','.

In [2]:
import pandas as pd

df = pd.read_csv('/content/datos_ejercicio_ventas.csv',delimiter=',')
df.to_csv('/content/datos_ejercicio_ventas.csv', sep=';', index=False)
df.head()



,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


**1.   NºActuals y Forecasts**

Para encontrar el nº de Actuals y Forecasts, nos quedamos con los datos que en la columna 'SCENARIO' sea igual a 'actual' o 'AI_forecast', y con shpe[0], obtenemos el número de filas de los valores actuales y los de predicción

In [3]:
num_actuals = df[df['SCENARIO'] == 'actual'].shape[0]


num_forecasts = df[df['SCENARIO'] == 'AI_forecast'].shape[0]

print(f"Número de Actuals: {num_actuals}")
print(f"Número de Forecasts: {num_forecasts}")

Número de Actuals: 900
Número de Forecasts: 17766


Encontramos muchas más predicciones que valores actuales en nuestro dataset.

**3.  Nº Países y de productos**

Para encontrar el nº de países y de productos, empleamos .unique() que nos cuenta los valores únicos de 'COUNTRY' y 'SUBBRAND'.

In [4]:

num_paises = df['COUNTRY'].nunique()
print(f'Número de países: {num_paises}')


num_productos = df['SUBBRAND'].nunique()
print(f'Número de productos: {num_productos}')

Número de países: 9
Número de productos: 6


**4.  Histórico de actuals/forecasts**
  - Primer dato de ventas real hasta el último
  - Primer dato de veentas forecast hasta el último

Lo primero que haremos será ordenar los datos por mes y año(esto nos servirá para encontrar la fecha más antigüa y la más reciente), luego nos quedamos con los datos que en la columna 'SCENARIO' sea igual a 'actual', y por último con .iloc[0] y .iloc[-1], cogemos los datos de la primera y última fila.

In [5]:
import pandas as pd

df = df.sort_values(by=['YEAR', 'MONTH'])

# Filtrar (actuals) y obtener las primeras y últimas fechas
actuals_data = df[df['SCENARIO'] == 'actual']
first_actual = actuals_data.iloc[0]
last_actual = actuals_data.iloc[-1]

# Filtrar (forecasts) y obtener las primeras y últimas fechas
forecast_data = df[df['SCENARIO'] == 'AI_forecast']
first_forecast = forecast_data.iloc[0]
last_forecast = forecast_data.iloc[-1]

# Mostrar los resultados
print("Primer dato de ventas real (actual): Año {}, Mes {}".format(first_actual['YEAR'], first_actual['MONTH']))
print("Último dato de ventas real (actual): Año {}, Mes {}".format(last_actual['YEAR'], last_actual['MONTH']))
print("Primer dato de predicción de ventas: Año {}, Mes {}".format(first_forecast['YEAR'], first_forecast['MONTH']))
print("Último dato de predicción de ventas: Año {}, Mes {}".format(last_forecast['YEAR'], last_forecast['MONTH']))

Primer dato de ventas real (actual): Año 2023, Mes 1
Último dato de ventas real (actual): Año 2024, Mes 8
Primer dato de predicción de ventas: Año 2023, Mes 1
Último dato de predicción de ventas: Año 2025, Mes 5


# PRACTICA 1




###**PREPARACIÓN DATOS:**




Lo primero que haremos para la preparación de datos es juntar 'MONTH' con 'YEAR' y convertirlo en datetime.

In [6]:
# Crear la columna 'Fecha' con columnas 'YEAR' y 'MONTH'
df['Fecha'] = pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))

# Eliminar las columnas 'YEAR' y 'MONTH' NO NECESARIAS
df.drop(columns=['YEAR', 'MONTH'], inplace=True)

# Mostrar
df.head()


,COUNTRY,SUBBRAND,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Fecha
34,Czech,Pepsi Max (L3),AI_forecast,AI_PF,2023.0,3.174244e+05,2023-01-01
395,Hungary,Pepsi Max (L3),AI_forecast,AI_PF,2023.0,5.050095e+05,2023-01-01
591,Portugal,Pepsi Regular (L3),AI_forecast,AI_PF,2023.0,1.692465e+05,2023-01-01
746,Great Britain,7up Free (L3),AI_forecast,AI_PF,2023.0,1.207824e+06,2023-01-01
770,Norway,7up (L3),AI_forecast,AI_PF,2023.0,6.636102e+03,2023-01-01


Convertir 'FORECAST_YEAR' y 'FORECAST' a formato datetime será más complicado, ya que será necesario crear un diccionario para convertir los periodos en meses y, posteriormente, mapear los valores de 'FORECAST' a meses.

In [7]:
# Crear un diccionario que mapea los valores de FORECAST a los meses
forecast_to_month = {
    'AI_P02F': 1,  # Enero
    'AI_P03F': 2,  # Febrero
    'AI_P04F': 3,  # Marzo
    'AI_P05F': 4,  # Abril
    'AI_P06F': 5,  # Mayo
    'AI_P07F': 6,  # Junio
    'AI_P08F': 7,  # Julio
    'AI_P09F': 8,  # Agosto
    'AI_P10F': 9,  # Septiembre
    'AI_P11F': 10, # Octubre
    'AI_P12F': 11, # Noviembre
    'AI_PF': 12    # Diciembre
}

# Usar map() para transformar los valores de la columna 'FORECAST' en meses
df['MONTH_FROM_FORECAST'] = df['FORECAST'].map(forecast_to_month)

# Mostrar
df.head()

,COUNTRY,SUBBRAND,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Fecha,MONTH_FROM_FORECAST
34,Czech,Pepsi Max (L3),AI_forecast,AI_PF,2023.0,3.174244e+05,2023-01-01,12.0
395,Hungary,Pepsi Max (L3),AI_forecast,AI_PF,2023.0,5.050095e+05,2023-01-01,12.0
591,Portugal,Pepsi Regular (L3),AI_forecast,AI_PF,2023.0,1.692465e+05,2023-01-01,12.0
746,Great Britain,7up Free (L3),AI_forecast,AI_PF,2023.0,1.207824e+06,2023-01-01,12.0
770,Norway,7up (L3),AI_forecast,AI_PF,2023.0,6.636102e+03,2023-01-01,12.0


Ahora sí podremos pasar 'FORECAST_YEAR' y 'FORECAST' a formato datetime y eliminamos las columnas 'MONTH', 'FORECAST_YEAR' y 'FORECAST', ya que no nos aporta información nueva.

In [8]:
#Pasar "MONTH_FROM_FORECAST" y "FORECAST_YEAR" a datetime
df["FECHA_FORECAST"] = pd.to_datetime(
    df[["MONTH_FROM_FORECAST", "FORECAST_YEAR"]]
    .assign(DAY=1)
    .rename(columns={"MONTH_FROM_FORECAST": "MONTH", "FORECAST_YEAR": "YEAR"})
)
df = df.drop(columns=['MONTH_FROM_FORECAST', 'FORECAST_YEAR','FORECAST'])
#Mostrar
df.head()

,COUNTRY,SUBBRAND,SCENARIO,AMOUNT,Fecha,FECHA_FORECAST
34,Czech,Pepsi Max (L3),AI_forecast,3.174244e+05,2023-01-01,2023-12-01
395,Hungary,Pepsi Max (L3),AI_forecast,5.050095e+05,2023-01-01,2023-12-01
591,Portugal,Pepsi Regular (L3),AI_forecast,1.692465e+05,2023-01-01,2023-12-01
746,Great Britain,7up Free (L3),AI_forecast,1.207824e+06,2023-01-01,2023-12-01
770,Norway,7up (L3),AI_forecast,6.636102e+03,2023-01-01,2023-12-01


Eliminamos duplicados para eliminar la información repetida y evitar que los datos se alteren.

In [9]:
df = df.drop_duplicates()

df.head()

,COUNTRY,SUBBRAND,SCENARIO,AMOUNT,Fecha,FECHA_FORECAST
34,Czech,Pepsi Max (L3),AI_forecast,3.174244e+05,2023-01-01,2023-12-01
395,Hungary,Pepsi Max (L3),AI_forecast,5.050095e+05,2023-01-01,2023-12-01
591,Portugal,Pepsi Regular (L3),AI_forecast,1.692465e+05,2023-01-01,2023-12-01
746,Great Britain,7up Free (L3),AI_forecast,1.207824e+06,2023-01-01,2023-12-01
770,Norway,7up (L3),AI_forecast,6.636102e+03,2023-01-01,2023-12-01


Comprobamos que se hayan eliminado bien los duplicados de nuestra base de datos.

In [10]:
duplicados = df[df.duplicated()]


count_duplicados = len(duplicados)

print("Total de filas duplicadas:", count_duplicados)
print(duplicados)


Total de filas duplicadas: 0
Empty DataFrame
Columns: [COUNTRY, SUBBRAND, SCENARIO, AMOUNT, Fecha, FECHA_FORECAST]
Index: []


Separamos los datos en datos actuales y de predicción.

In [11]:
# obtener los 'actual'
df_actuals = df[df['SCENARIO'] == 'actual'].reset_index(drop=True)

# 'AI_forecast'
df_ia = df[df['SCENARIO'] == 'AI_forecast']
df_actuals = df_actuals.drop(columns=['SCENARIO','FECHA_FORECAST'])
df_ia = df_ia.drop(columns=['SCENARIO'])

In [12]:
print("DataFrame de datos actuales:")
df_actuals.head()

DataFrame de datos actuales:


,COUNTRY,SUBBRAND,AMOUNT,Fecha
0,Spain,Pepsi Max (L3),8.327406e+04,2023-01-01
1,Hungary,7up (L3),6.464034e+04,2023-01-01
2,Netherlands,Pepsi Max (L3),6.868530e+05,2023-01-01
3,Norway,Pepsi Max (L3),2.521704e+06,2023-01-01
4,Portugal,7up Free (L3),7.207855e+04,2023-01-01


In [13]:
print("\nDataFrame de predicciones:")
df_ia.head()


DataFrame de predicciones:


,COUNTRY,SUBBRAND,AMOUNT,Fecha,FECHA_FORECAST
34,Czech,Pepsi Max (L3),3.174244e+05,2023-01-01,2023-12-01
395,Hungary,Pepsi Max (L3),5.050095e+05,2023-01-01,2023-12-01
591,Portugal,Pepsi Regular (L3),1.692465e+05,2023-01-01,2023-12-01
746,Great Britain,7up Free (L3),1.207824e+06,2023-01-01,2023-12-01
770,Norway,7up (L3),6.636102e+03,2023-01-01,2023-12-01


## 1.Como se distribuyen las ventas realizadas:


1.   En cada país
2.   En cada mes y año
3.   En cada marca



###**1. En cada país:**




Preparamos los datos para analizar las distribuciones de las ventas en cada país. Lo que haremos será agrupar los litros vendidos por país

In [14]:
df_PA_act = df_actuals.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
df_PA_act = df_PA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})  # Renombrar la columna 'AMOUNT' a 'TOTAL_AMOUNT'

In [15]:
print(df_PA_act)

         COUNTRY  TOTAL_AMOUNT
0          Czech  3.535164e+07
1        Denmark  5.659668e+07
2  Great Britain  3.347786e+08
3        Hungary  4.153991e+07
4          Italy  4.345404e+07
5    Netherlands  6.395943e+07
6         Norway  5.121406e+07
7       Portugal  3.488807e+07
8          Spain  8.131266e+06


Haremos un gráfico de barras para observar las distribuciones de ventas en cada país.

In [16]:
import plotly.express as px
# Crear la gráfica de barras
fig = px.bar(df_PA_act,
             x='COUNTRY',  # Países en el eje X
             y='TOTAL_AMOUNT',   # Montos en el eje Y
             title='Distribución de ventas en cada País',  # Título de la gráfica
             labels={'TOTAL_AMOUNT': 'CANTIDAD EN L', 'COUNTRY': 'Países'},  # Etiquetas de los ejes
             color='COUNTRY',  # Puedes asignar colores diferentes a cada país
             color_discrete_sequence=px.colors.qualitative.Set1  # Color discreto para las barras
            )

# Actualizar el formato del eje Y para mostrar valores en formato científico
fig.update_layout(
    yaxis_tickformat='.2e'  # El formato '.2e' muestra los números en notación científica con 2 decimales
)

# Mostrar la gráfica
fig.show()

Podemos observar que el país con más ventas y por mucho es Gran Bretaña, y el que menos ventas tiene es España, quitando Gran Bretaña los datos están más o menos equilibrados.

###**2.En cada mes y año**

Preparamos los datos para analizar las distribuciones de las ventas en cada mes y año. Lo que haremos será agrupar los litros vendidos por mes y año.

In [17]:
df_FA_act = df_actuals.groupby('Fecha')['AMOUNT'].sum().reset_index()
df_FA_act = df_FA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})


In [18]:
df_FA_act.head()

,Fecha,TOTAL_AMOUNT
0,2023-01-01,2.808880e+07
1,2023-02-01,2.610437e+07
2,2023-03-01,3.593163e+07
3,2023-04-01,3.085725e+07
4,2023-05-01,3.690245e+07


Haremos un gráfico de barras para ver la distribuciones de ventas en cada mes y año.

In [19]:
# Crear la gráfica de barras
fig = px.bar(df_FA_act,
             x='Fecha',  # Fechas en el eje X
             y='TOTAL_AMOUNT',  # Montos en el eje Y
             title='Distribución de ventas realizadas en cada Mes y año',  # Título de la gráfica
             labels={'TOTAL_AMOUNT': 'Cantidad en L', 'Fecha': 'Fecha'},  # Etiquetas de los ejes
            )

# Actualizar el formato del eje Y para mostrar los valores en formato científico
fig.update_layout(
    yaxis_tickformat='.2e',  # Mostrar en formato científico con 2 decimales
)

# Mostrar la gráfica
fig.show()

Observamos que los datos están muy equilibrados, destacando el mes de agosto de 2023 y 2024, y que en enero y febrero de 2023 y 2024 obtenemos menos ventas.

###**3.En cada marca**

Preparamos los datos para analizar las distribuciones de las ventas en cada marca. Lo que haremos será agrupar los litros vendidos por marca.

In [20]:
df_MA_act = df_actuals.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
df_MA_act = df_MA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})

In [21]:
df_MA_act.head()

,SUBBRAND,TOTAL_AMOUNT
0,7up (L3),1.876851e+07
1,7up Free (L3),3.427630e+07
2,Lipton (L3),7.286515e+07
3,Mountain Dew (L3),7.474391e+06
4,Pepsi Max (L3),3.966987e+08


Haremos un gráfico de tarta para observar las distribuciones de ventas de cada marca.

In [22]:
fig = px.pie(df_MA_act,
             names='SUBBRAND',  # Categorías en el gráfico (SUBBRAND)
             values='TOTAL_AMOUNT',  # Valores correspondientes a cada categoría (TOTAL_AMOUNT)
             title='Distribución de ventas realizadas en cada marca',  # Título del gráfico
             color='SUBBRAND',  # Asigna colores diferentes a cada categoría
             color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores discreta
            )

# Mostrar la gráfica
fig.show()

Vemos que hay una gran diferencia entre Pepsi Max y el resto siguiendole Pepsi Regular, con esto sacamos que la mejor marca de nuestra base de datos es Pepsi, obteniendo un casi 80% entre Pepsi Max y Pepsi Regular.

##2.Cual es la tendencia y estacionalidad de:


1.   Todas las ventas del país con menos ventas
2.   La marca con más ventas




###**1.Todas las ventas del país con menos ventas**

Ordenamos el el dataframe de países (df_PA_act) de menor a mayor

In [23]:
# Ordenar el DataFrame de menor a mayor
df_PA_sorted_act = df_PA_act.sort_values(by='TOTAL_AMOUNT', ascending=True)

# Mostrar
print(df_PA_sorted_act)

         COUNTRY  TOTAL_AMOUNT
8          Spain  8.131266e+06
7       Portugal  3.488807e+07
0          Czech  3.535164e+07
3        Hungary  4.153991e+07
4          Italy  4.345404e+07
6         Norway  5.121406e+07
1        Denmark  5.659668e+07
5    Netherlands  6.395943e+07
2  Great Britain  3.347786e+08


Como podemos comprobar, el país con menos ventas es España. Esto ya lo pudimos observar en la gráfica de distribuciones de ventas de cada país.


Preparamos los datos para estudiar la tendencia y estacionalidad de todas las ventas de España. Lo que haremos filtrar los datos para que solo me quede el país España, eliminamos la columna 'SUBBRAND', que ya no la necesitaremos, agrupamos 'COUNTRY' y 'Fecha' y sumamos los respectivos 'AMOUNT'.

In [24]:
df_espana = df_actuals[df_actuals['COUNTRY'] == 'Spain']  # Filtrar para España
df_espana = df_espana.drop(columns=['SUBBRAND'])  # Eliminar la columna 'SUBBRAND'
df_espana = df_espana.groupby(['COUNTRY', 'Fecha'])['AMOUNT'].sum().reset_index()  # Agrupar por 'COUNTRY' y 'Fecha'
df_espana = df_espana.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})  # Renombrar la columna 'AMOUNT' a 'TOTAL_AMOUNT'
print(df_espana)  # Imprimir el DataFrame resultante


   COUNTRY      Fecha   TOTAL_AMOUNT
0    Spain 2023-01-01  325272.938930
1    Spain 2023-02-01  295588.379605
2    Spain 2023-03-01  399710.273110
3    Spain 2023-04-01  328035.257874
4    Spain 2023-05-01  360751.156280
5    Spain 2023-06-01  439198.586290
6    Spain 2023-07-01  496545.236120
7    Spain 2023-08-01  564336.120820
8    Spain 2023-09-01  420041.980591
9    Spain 2023-10-01  468425.841760
10   Spain 2023-11-01  418006.199049
11   Spain 2023-12-01  394147.408590
12   Spain 2024-01-01  349696.533142
13   Spain 2024-02-01  352899.035652
14   Spain 2024-03-01  332109.256233
15   Spain 2024-04-01  376678.368897
16   Spain 2024-05-01  356759.011786
17   Spain 2024-06-01  408967.746367
18   Spain 2024-07-01  506234.362782
19   Spain 2024-08-01  537862.283643


Haremos un gráfico de línea para observar la tendencia y estacionalidad de todas las ventas de España.

In [25]:

fig = px.line(df_espana,
              x='Fecha',  # Fechas en el eje X
              y='TOTAL_AMOUNT',  # Montos en el eje Y
              title='Cantidad vendida en España',  # Título de la gráfica
              labels={'AMOUNT': 'Cantidad vendida en L', 'Fecha': 'Fecha'},  # Etiquetas de los ejes
              line_group='COUNTRY',  # Agrupar por país (aunque en este caso solo hay un país)
              color='COUNTRY'  # Asigna colores a las líneas (aunque solo hay una línea para España)
             )

# Mostrar la gráfica
fig.show()

In [26]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose

# Realizar la descomposición de la serie temporal con un periodo de 6
decomposition = seasonal_decompose(df_espana['TOTAL_AMOUNT'], model='additive', period=6)

# Extraer las componentes: tendencia, estacionalidad y residuales
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Crear las gráficas
# Gráfica de la serie temporal original
fig = px.line(df_espana, x=df_espana.index, y='TOTAL_AMOUNT', title='Ventas en España (Original)',
              labels={'TOTAL_AMOUNT': 'Monto Total'}, markers=True)
fig.show()

# Gráfica de la tendencia
fig_trend = px.line(x=df_espana.index, y=trend, title='Tendencia de las ventas',
                    labels={'value': 'Tendencia', 'index': 'Fecha'})
fig_trend.show()

# Gráfica de la estacionalidad
fig_seasonal = px.line(x=df_espana.index, y=seasonal, title='Estacionalidad de las ventas',
                       labels={'value': 'Estacionalidad', 'index': 'Fecha'})
fig_seasonal.show()

# Gráfica de los residuos (errores no explicados)
fig_residual = px.line(x=df_espana.index, y=residual, title='Residuales de las ventas',
                       labels={'value': 'Residuales', 'index': 'Fecha'})
fig_residual.show()


Nos encontramos con una tendencia positiva al hasta julio 2023, luego esa tendencia baja hasta marzo de 2024, y vuelve a subir al final del periodo. En cuanto a la estacionalidad, podemos concluir que hay una cierta estacionalidad en los meses marzo y julio con picos repetitivos en los años 2023 y 2024. Lo que podemos concluir que en España afecta las ventas según la época del año.


###**2.La marca con más ventas**

preparamos los datos para estudiar la tendencia y estacionalidad de la marca con más ventas. Lo que haremos filtrar los datos para que solo me quede PepsiMax que como vimos en la gráfica de tartas es la marca con más ventas.

In [27]:
# Filtrar para 'Pepsi Max (L3)'
# y seleccionar solo las columnas 'SUBBRAND' y 'TOTAL_AMOUNT'
df_PepsiMax = df_actuals[df_actuals['SUBBRAND'] == 'Pepsi Max (L3)']
df_PepsiMax = df_PepsiMax.drop(columns=['COUNTRY'])  # Eliminar la columna 'SUBBRAND'
df_PepsiMax = df_PepsiMax.groupby(['SUBBRAND', 'Fecha'])['AMOUNT'].sum().reset_index()  # Agrupar por 'COUNTRY' y 'Fecha'
df_PepsiMax = df_PepsiMax.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})  # Renombrar la columna 'AMOUNT' a 'TOTAL_AMOUNT'


# Mostrar
df_PepsiMax.head()


,SUBBRAND,Fecha,TOTAL_AMOUNT
0,Pepsi Max (L3),2023-01-01,1.485655e+07
1,Pepsi Max (L3),2023-02-01,1.546332e+07
2,Pepsi Max (L3),2023-03-01,2.087355e+07
3,Pepsi Max (L3),2023-04-01,1.793494e+07
4,Pepsi Max (L3),2023-05-01,2.232545e+07


Haremos un gráfico de línea para estudiar la tendencia y estacionaridad de la marca más vendida.

In [28]:
import plotly.express as px

# Crear la gráfica de línea
fig = px.line(df_PepsiMax,
              x='Fecha',  # Columna en el eje X
              y='TOTAL_AMOUNT',  # Columna en el eje Y
              title='Ventas de Pepsi Max (L3) a lo largo del tiempo',  # Título de la gráfica
              labels={'TOTAL_AMOUNT': 'Cantidad vendida Total', 'Fecha': 'Fecha'},  # Etiquetas de los ejes
              markers=True)  # Mostrar los puntos de datos en la línea

# Mostrar la gráfica
fig.show()


In [29]:
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

# Establecer 'Fecha' como índice
df_PepsiMax.set_index('Fecha', inplace=True)

# Realizar la descomposición de la serie temporal con periodo de 6 (para ajustarse a tus 20 observaciones)
decomposition = seasonal_decompose(df_PepsiMax['TOTAL_AMOUNT'], model='additive', period=6)

# Extraer las componentes
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Crear la gráfica de la tendencia
fig_trend = px.line(x=df_PepsiMax.index, y=trend,
                    title='Tendencia de las ventas de Pepsi Max (L3)',
                    labels={'value': 'Tendencia', 'index': 'Fecha'})

# Crear la gráfica de la estacionalidad
fig_seasonal = px.line(x=df_PepsiMax.index, y=seasonal,
                       title='Estacionalidad de las ventas de Pepsi Max (L3)',
                       labels={'value': 'Estacionalidad', 'index': 'Fecha'})

# Crear la gráfica de los residuales (errores no explicados)
fig_residual = px.line(x=df_PepsiMax.index, y=residual,
                       title='Residuales de las ventas de Pepsi Max (L3)',
                       labels={'value': 'Residuales', 'index': 'Fecha'})

# Mostrar las gráficas
fig_trend.show()
fig_seasonal.show()
fig_residual.show()


Hay una tendencia general positiva, con algunas fluctuaciones(enero 2024), con una estacionalidad casi inexistente, con lo que concluimos que las ventas de Pepsi Max están en crecimiento, pero no necesariamente están siendo influenciadas por factores estacionales.

##3.Cuales son las predicciones hechas en España y como de buenas son.

df_espana_ia la creamos para filtrar los datos predichos de España y agruparlos por 'COUNTRY', 'Fecha' y 'AMOUNT'. El dataset con los valores de España actuales ya esta creada(df_espana)

In [30]:
df_espana_ia=df_ia[df_ia['COUNTRY'] == 'Spain']
df_espana_ia = df_espana_ia.drop(columns=['SUBBRAND'])
df_espana_ia = df_espana_ia.groupby(['COUNTRY', 'Fecha'])['AMOUNT'].sum().reset_index()
df_espana_ia = df_espana_ia.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})
df_espana_ia.head()


,COUNTRY,Fecha,TOTAL_AMOUNT
0,Spain,2023-01-01,3.227921e+05
1,Spain,2023-02-01,6.341964e+05
2,Spain,2023-03-01,9.632081e+05
3,Spain,2023-04-01,1.693046e+06
4,Spain,2023-05-01,2.523458e+06


Creamos un dataset que junte los datos actuales y los predichos de España por fecha, para poder compararlos.

In [31]:
# Filtrar las fechas que coinciden entre los dos DataFrames
merged_df = pd.merge(df_espana_ia[['Fecha', 'TOTAL_AMOUNT']], df_espana[['Fecha', 'TOTAL_AMOUNT']],
                     on='Fecha', suffixes=('_Predicted', '_Actual'))
merged_df.head()

,Fecha,TOTAL_AMOUNT_Predicted,TOTAL_AMOUNT_Actual
0,2023-01-01,3.227921e+05,325272.938930
1,2023-02-01,6.341964e+05,295588.379605
2,2023-03-01,9.632081e+05,399710.273110
3,2023-04-01,1.693046e+06,328035.257874
4,2023-05-01,2.523458e+06,360751.156280


Creamos una gráfica de barras para poder comparar los datos reales con los predichos.

In [32]:
import pandas as pd
import plotly.express as px



# Crear la gráfica de barras
fig = px.bar(merged_df,
             x='Fecha',  # Fechas en el eje X
             y=['TOTAL_AMOUNT_Predicted', 'TOTAL_AMOUNT_Actual'],  # Predicciones y valores actuales en el eje Y
             title='Comparación de Predicciones vs Valores Actuales',  # Título de la gráfica
             labels={'TOTAL_AMOUNT_Predicted': 'Predicciones', 'TOTAL_AMOUNT_Actual': 'Valores Actuales', 'Fecha': 'Fecha'},
             barmode='group',  # Las barras de predicciones y actuales estarán agrupadas
             width=800,  # Ancho de la gráfica
             height=600)  # Alto de la gráfica

# Mostrar la gráfica
fig.show()


Vemos que los valores predichos suelen ser mucho más altos que los valores actuales, por lo que no estamos ante un buen modelo de predicción,  porque los datos predichos se desvían mucho de los datos reles.